#### 1 Importar bibliotecas

In [2]:
import pandas as pd
from hdbcli import dbapi
import datetime
import time
import time ; from datetime import date, timedelta #biblioteca para trabalhar com tempo, datas...
import os
import warnings
warnings.filterwarnings('ignore')
import openpyxl

In [4]:
data_hoje = date.today() - timedelta(0) 
data_ontem = date.today() - timedelta(1) 
data_amanha = date.today() + timedelta(1) 
ano_hoje = str(data_hoje.year)
dia_hoje = str(data_hoje.day)
mes_hoje = str(data_hoje.month)
if len(dia_hoje) == 1:
    dia_hoje = '0' + dia_hoje
if len(mes_hoje) == 1:
    mes_hoje = '0' + mes_hoje
    
if str(data_ontem.day) == '7' and str(data_ontem.month) == '9' and str(data_ontem.year) == '2022':
    print('dia')

dia


In [46]:
aba_metas = "Base (Planejado)"
aba_orcamento = "Orçamento"
ano_inteiro_hoje = str(date.today().year)
ano_hoje = str(date.today().year)[2:]
mes_hoje = int(date.today().month)
mes_anterior = int(date.today().month) - 1
mes_anterior_2 = int(date.today().month) - 2
if len(str(mes_hoje)) == 1:
    mes_hoje_str = '0' + str(mes_hoje)
if len(str(mes_anterior)) == 1:
    mes_anterior_str = '0' + str(mes_anterior)
if len(str(mes_anterior_2)) == 1:
    mes_anterior_2_str = '0' + str(mes_anterior_2)
filtro_mes_ref = int(ano_inteiro_hoje + mes_hoje_str)
filtro_mes_ref_orcamento = int(ano_inteiro_hoje + mes_anterior_str)
filtro_mes_ref_orcamento_2 = int(ano_inteiro_hoje + mes_anterior_2_str)

if mes_hoje == 1:
    nome_arquivo = "Dashboard Cobranca Jan"+ano_hoje
elif mes_hoje == 2:
    nome_arquivo = "Dashboard Cobranca Fev"+ano_hoje
    nome_arquivo_anterior = "Dashboard Cobranca Jan"+ano_hoje
elif mes_hoje == 3:
    nome_arquivo = "Dashboard Cobranca Mar"+ano_hoje
    nome_arquivo_anterior = "Dashboard Cobranca Fev"+ano_hoje
elif mes_hoje == 4:
    nome_arquivo = "Dashboard Cobranca Abr"+ano_hoje
    nome_arquivo_anterior = "Dashboard Cobranca Mar"+ano_hoje
elif mes_hoje == 5:
    nome_arquivo = "Dashboard Cobranca Mai"+ano_hoje
    nome_arquivo_anterior = "Dashboard Cobranca Abr"+ano_hoje
elif mes_hoje == 6:
    nome_arquivo = "Dashboard Cobranca Jun"+ano_hoje
    nome_arquivo_anterior = "Dashboard Cobranca Mai"+ano_hoje
elif mes_hoje == 7:
    nome_arquivo = "Dashboard Cobranca Jul"+ano_hoje
    nome_arquivo_anterior = "Dashboard Cobranca Jun"+ano_hoje
elif mes_hoje == 8:
    nome_arquivo = "Dashboard Cobranca Ago"+ano_hoje+"p"
    nome_arquivo_anterior = "Dashboard Cobranca Jul"+ano_hoje
elif mes_hoje == 9:
    nome_arquivo = "Dashboard Cobranca Set"+ano_hoje
    nome_arquivo_anterior ="Dashboard Cobranca Ago"+ano_hoje+"p"
elif mes_hoje == 10:
    nome_arquivo = "Dashboard Cobranca Out"+ano_hoje
    nome_arquivo_anterior = "Dashboard Cobranca Set"+ano_hoje
elif mes_hoje == 11:
    nome_arquivo = "Dashboard Cobranca Nov"+ano_hoje
    nome_arquivo_anterior = "Dashboard Cobranca Out"+ano_hoje
elif mes_hoje == 12:
    nome_arquivo = "Dashboard Cobranca Dez"+ano_hoje
    nome_arquivo_anterior = "Dashboard Cobranca Nov"+ano_hoje

arquivo_dashboard = r'\\BRNEP242\Publico\DDT\PR\SCL\CCO\COGC\Equipe\CRRC\2. Plano de Recuperação de Créditos\\{}.xlsb'.format(nome_arquivo)

try:
    df_metas = pd.read_excel(arquivo_dashboard,sheet_name = aba_metas)
    df_orcamento = pd.read_excel(arquivo_dashboard,sheet_name = aba_orcamento)
except Exception as e:
    print(e)
    try: 
        arquivo_dashboard = r'\\BRNEP242\Publico\DDT\PR\SCL\CCO\COGC\Equipe\CRRC\2. Plano de Recuperação de Créditos\\{}.xlsb'.format(nome_arquivo_anterior)
        df_metas = pd.read_excel(arquivo_dashboard,sheet_name = aba_metas)
        df_orcamento = pd.read_excel(arquivo_dashboard,sheet_name = aba_orcamento)
        filtro_mes_ref_orcamento = filtro_mes_ref_orcamento_2
    except Exception as e:
        print(e)
        
df_metas = df_metas[['MESREF','SETOR','UTD','TURMA','ATIVIDADE','QTD']]
df_metas = pd.DataFrame(df_metas.groupby(['MESREF','SETOR','UTD','TURMA','ATIVIDADE'])['QTD'].sum()).reset_index()

df_metas.rename(columns = {'ATIVIDADE' : 'CARTEIRA','QTD':'QTD_META'}, inplace = True)
df_metas['QTD_META'] = df_metas['QTD_META'].fillna(0)
df_metas['QTD_META'] = df_metas['QTD_META'].astype(int)
df_metas = df_metas[((df_metas['MESREF'] == filtro_mes_ref))]
df_metas.loc[(df_metas.TURMA == "PRÓPRIA"), "TURMA"] = "STC"
df_metas = df_metas[['UTD','TURMA','CARTEIRA','QTD_META']]


df_orcamento = df_orcamento[['MÊS','SETOR','UTD','PLANEJADO','REALIZADO']]
df_orcamento = df_orcamento[((df_orcamento['MÊS'] <= filtro_mes_ref_orcamento))]
df_orcamento = pd.DataFrame(df_orcamento.groupby(['SETOR','UTD'])['PLANEJADO','REALIZADO'].sum()).reset_index()
df_orcamento['PERCENT'] = df_orcamento['REALIZADO']/df_orcamento['PLANEJADO']
df_orcamento['PERCENT'] = df_orcamento['PERCENT'].astype(int)
df_orcamento['TURMA'] = 'EPS'
df_orcamento = df_orcamento[['UTD','TURMA','PERCENT']]

[Errno 2] No such file or directory: '\\\\BRNEP242\\Publico\\DDT\\PR\\SCL\\CCO\\COGC\\Equipe\\CRRC\\2. Plano de Recuperação de Créditos\\\\Dashboard Cobranca Set22.xlsb'


In [41]:
df_orcamento.info()
display(df_orcamento)
df_orcamento = df_orcamento[['MÊS','SETOR','UTD','PLANEJADO','REALIZADO']]
df_orcamento = df_orcamento[((df_orcamento['MÊS'] == filtro_mes_ref_orcamento))]
df_orcamento = pd.DataFrame(df_orcamento.groupby(['UTD'])['PLANEJADO','REALIZADO'].sum()).reset_index()
df_orcamento['PERCENT'] = df_orcamento['REALIZADO']/df_orcamento['PLANEJADO']
#
#print(filtro_mes_ref)
#total_freq = len(df_metas)
#for i in range(total_freq):
#    if df_metas['MESREF'][i] == 202212:
#        print(df_metas['MESREF'][i])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 540 entries, 0 to 539
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   REGIONAL     540 non-null    object 
 1   SETOR        540 non-null    object 
 2   UTD          540 non-null    object 
 3   MÊS          540 non-null    int64  
 4   PLANEJADO    540 non-null    float64
 5   REALIZADO    540 non-null    float64
 6   OBS          540 non-null    object 
 7   Unnamed: 7   0 non-null      float64
 8   Unnamed: 8   0 non-null      float64
 9   Unnamed: 9   0 non-null      float64
 10  Unnamed: 10  10 non-null     float64
dtypes: float64(6), int64(1), object(4)
memory usage: 46.5+ KB


,REGIONAL,SETOR,UTD,MÊS,PLANEJADO,REALIZADO,OBS,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,NORTE,CENTRO,CONCEICAO DO COITE,202201,-58600.000000,-58600.00,REV2',NaN,NaN,NaN,NaN
1,NORTE,CENTRO,CONCEICAO DO COITE,202202,47567.550000,47567.55,REV2',NaN,NaN,NaN,NaN
2,NORTE,CENTRO,CONCEICAO DO COITE,202203,71463.840000,71463.84,REV2',NaN,NaN,NaN,NaN
3,NORTE,CENTRO,CONCEICAO DO COITE,202204,39252.240000,39252.24,REV2',NaN,NaN,NaN,NaN
4,NORTE,CENTRO,CONCEICAO DO COITE,202205,14609.880000,14609.88,REV2',NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
535,SUL,SUL,TEIXEIRA DE FREITAS,202208,134601.056997,0.00,REV2',NaN,NaN,NaN,NaN
536,SUL,SUL,TEIXEIRA DE FREITAS,202209,92596.761397,0.00,REV2',NaN,NaN,NaN,NaN
537,SUL,SUL,TEIXEIRA DE FREITAS,202210,86973.505490,0.00,REV2',NaN,NaN,NaN,NaN
538,SUL,SUL,TEIXEIRA DE FREITAS,202211,87889.941355,0.00,REV2',NaN,NaN,NaN,NaN


In [42]:
df_orcamento_TEMP = pd.DataFrame(df_orcamento.groupby(['UTD'])['PLANEJADO','REALIZADO'].sum()).reset_index()

In [10]:
date1 = "05/09/2022"
date2 = "01/01/2016"
newdate1 = time.strptime(date1, "%d/%m/%Y")
newdate2 = time.strptime(date2, "%d/%m/%Y")
print(newdate1)
dia_hoje = date.today()
if dia_hoje == newdate1:
    print(dia_hoje)
#print(dia_hoje)

time.struct_time(tm_year=2022, tm_mon=9, tm_mday=5, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=0, tm_yday=248, tm_isdst=-1)


In [35]:
usuario = 'clb344173'
usuario_u = 'u'+usuario[3:]
print(usuario_u)

nome_arquivo = "Defasagem 2022"
arquivo_defasagem = r'C:\\Users\\{}\\OneDrive - IBERDROLA S.A\\GESTÃO DA DEMANDA - PROGRAMAÇÃO COMERCIAL\\{}.xlsx'.format(usuario_u,nome_arquivo)
data_hoje = date.today() - timedelta(0)
ano_hoje = str(data_hoje.year)
dia_hoje = str(data_hoje.day)
mes_hoje = str(data_hoje.month)
if len(dia_hoje) == 1:
    dia_hoje = '0' + dia_hoje
if len(mes_hoje) == 1:
    mes_hoje = '0' + mes_hoje




try:
    aba = dia_hoje +"." + mes_hoje 
    df_pedido_defasagem = pd.read_excel(arquivo_defasagem,sheet_name = aba)
except Exception as e:
    print(e)
    try: 
        aba = dia_hoje +"." + mes_hoje +" " 
        df_pedido_defasagem = pd.read_excel(arquivo_defasagem,sheet_name = aba)
    except Exception as e:
        print(e)
        try:
            aba = dia_hoje +"." + mes_hoje +"  " 
            df_pedido_defasagem = pd.read_excel(arquivo_defasagem,sheet_name = aba)
        except Exception as e:
            print(e)

df_pedido_defasagem = df_pedido_defasagem[['SETOR','UTD','ID','TIPO','QTD. TURMAS','QTD. NOTAS/TURMA','OBS']]
df_pedido_defasagem.rename(columns = {'ID' : 'ZONA','TIPO':'TIPO_SERVICO','QTD. TURMAS':'QTD_TURMAS','QTD. NOTAS/TURMA':'QTD_NOTAS','OBS':'JUSTIFICATIVA'}, inplace = True)
df_pedido_defasagem['QTD_TURMAS'] = df_pedido_defasagem['QTD_TURMAS'].fillna(0)
df_pedido_defasagem['QTD_NOTAS'] = df_pedido_defasagem['QTD_NOTAS'].fillna(0)
df_pedido_defasagem['TIPO_SERVICO'] = df_pedido_defasagem['TIPO_SERVICO'].fillna('-')
df_pedido_defasagem['JUSTIFICATIVA'] = df_pedido_defasagem['JUSTIFICATIVA'].fillna('-')
df_pedido_defasagem['QTD_TURMAS'] = df_pedido_defasagem['QTD_TURMAS'].astype(int)
df_pedido_defasagem['QTD_NOTAS'] = df_pedido_defasagem['QTD_NOTAS'].astype(int)
df_pedido_defasagem['ZONA']  = df_pedido_defasagem['ZONA'].astype(str).str.pad(4,'left','0')
df_pedido_defasagem['ACAO'] = ""
df_pedido_defasagem['CARTEIRA'] = ""
df_pedido_defasagem['TURMA'] = "STC"
df_pedido_defasagem.loc[((df_pedido_defasagem.JUSTIFICATIVA == "-") &(df_pedido_defasagem.TIPO_SERVICO == "-") & (df_pedido_defasagem.QTD_TURMAS == 0) & (df_pedido_defasagem.QTD_NOTAS == 0)), "ACAO"] = "APAGAR"
df_pedido_defasagem = df_pedido_defasagem[((df_pedido_defasagem['ACAO'] != "APAGAR"))]
df_pedido_defasagem.loc[(df_pedido_defasagem.TIPO_SERVICO == "CA"), "CARTEIRA"] = "RECORTE"
df_pedido_defasagem.loc[(df_pedido_defasagem.TIPO_SERVICO == "CS"), "CARTEIRA"] = "CORTE"
df_pedido_defasagem['DEFASAGEM_PROGRAMACAO'] = df_pedido_defasagem['QTD_TURMAS'] * df_pedido_defasagem['QTD_NOTAS']
df_pedido_defasagem = df_pedido_defasagem.dropna().reset_index(drop=True)        
      
display(df_pedido_defasagem)

u344173
Worksheet named '28.08' not found
Worksheet named '28.08 ' not found


,SETOR,UTD,ZONA,TIPO_SERVICO,QTD_TURMAS,QTD_NOTAS,JUSTIFICATIVA,ACAO,CARTEIRA,TURMA,DEFASAGEM_PROGRAMACAO
0,CENTRO SUL,AMARGOSA,06.0,CA,1,18,Turma extra,,RECORTE,STC,18
1,CENTRO SUL,AMARGOSA,11.0,CA,2,15,Turma extra,,RECORTE,STC,30
2,OESTE,BARREIRAS,22.0,CS,1,20,Turma extra,,CORTE,STC,20
3,SUDOESTE,GUANAMBI,138.0,CS,1,15,Complementar roteiro,,CORTE,STC,15
4,SUDOESTE,GUANAMBI,142.0,CS,1,25,Gavião - Colaborador sozinho,,CORTE,STC,25
5,SUDOESTE,GUANAMBI,152.0,CS,1,15,Complementar roteiro,,CORTE,STC,15
6,CENTRO OESTE,IRECE,192.0,CS,1,18,Complementar roteiro,,CORTE,STC,18
7,CENTRO OESTE,IRECE,194.0,CS,1,18,Complementar roteiro,,CORTE,STC,18
8,CENTRO OESTE,IRECE,200.0,CA,1,20,Complementar roteiro,,RECORTE,STC,20
9,CENTRO OESTE,IRECE,195.0,CS,1,15,Complementar roteiro,,CORTE,STC,15


#### 2 Configurações

In [9]:
data_hoje = date.today()
data_insercao = data_hoje
dia_semana = datetime.datetime.today().weekday()
if dia_semana == 0: #segunda
    data_pedido = date.today() - timedelta(3)
elif dia_semana == 6:
    data_pedido = date.today() - timedelta(3)
elif dia_semana == 5:
    data_pedido = date.today() - timedelta(2)
else:
    data_pedido = date.today() - timedelta(1)
def color_survived(val):
    if val > 7:
        color = 'red' 
    elif val > 3:
        color = 'green' 
    return f'background-color: {color}'




In [19]:
usuario_hana = "CLB352618"
senha_hana = "PCtrab40#"
usuario = "u344173"
nome_arquivo = "Defasagem 2022"
arquivo_defasagem = r'C:\\Users\\{}\\OneDrive - IBERDROLA S.A\\GESTÃO DA DEMANDA - PROGRAMAÇÃO COMERCIAL\\{}.xlsx'.format(usuario,nome_arquivo)
data_hoje = date.today()
ano_hoje = str(data_hoje.year)
dia_hoje = str(data_hoje.day)
mes_hoje = str(data_hoje.month)
if len(dia_hoje) == 1:
    dia_hoje = '0' + dia_hoje
if len(mes_hoje) == 1:
    mes_hoje = '0' + mes_hoje

#aba = dia_hoje +"." + mes_hoje +"  " 
#aba = dia_hoje +"." + mes_hoje 
aba = '28.08  '
nome_aco = 'Acompanhamento Diário 2022 - Cobrança_' + dia_hoje + mes_hoje 
arquivo_aco = r'\\BRNEP242\Publico\DDT\PR\SCL\CCO\COGC\Equipe\CRRC\22. ACO\2022\{}.xlsb'.format(nome_aco)
print(arquivo_aco)

\\BRNEP242\Publico\DDT\PR\SCL\CCO\COGC\Equipe\CRRC\22. ACO\2022\Acompanhamento Diário 2022 - Cobrança_2808.xlsb


#### Coletar dados ACO

In [3]:
condicao = 'nao'
if condicao == 'sim':
    aco_visitado_mes = pd.read_excel(arquivo_aco,sheet_name = 'BASE_VISITADOS')
    aco_visitado_mes = aco_visitado_mes[['UTD40','TURMA','TIPO','QTD']]
    aco_visitado_mes.rename(columns = {'UTD40' : 'UTD','TIPO':'CARTEIRA','QTD':'QTD_VISITAS'}, inplace = True)
    aco_visitado_mes.loc[((aco_visitado_mes.TURMA == "PRÓPRIA") ), "TURMA"] = "STC"
    aco_visitado_mes = aco_visitado_mes[((aco_visitado_mes['TURMA'] == "EPS") | (aco_visitado_mes['TURMA'] == "STC"))]
    aco_visitado_mes.loc[((aco_visitado_mes.CARTEIRA == "Corte") ), "CARTEIRA"] = "CORTE"
    aco_visitado_mes.loc[((aco_visitado_mes.CARTEIRA == "Baixa") ), "CARTEIRA"] = "BAIXA"
    aco_visitado_mes.loc[((aco_visitado_mes.CARTEIRA == "Recorte") ), "CARTEIRA"] = "RECORTE"

    aco_meta_mes = pd.read_excel(arquivo_aco,sheet_name = 'META COBRANÇA',skiprows = 1)
    aco_meta_mes = aco_meta_mes[['UTD','EXECUTOR','TIPO','QTD']]
    aco_meta_mes.rename(columns = {'EXECUTOR' : 'TURMA','TIPO':'CARTEIRA','QTD':'QTD_META'}, inplace = True)
    aco_meta_mes.loc[((aco_meta_mes.TURMA == "PRÓPRIA") ), "TURMA"] = "STC"
    aco_meta_mes = aco_meta_mes[((aco_meta_mes['TURMA'] == "EPS") | (aco_visitado_mes['TURMA'] == "STC"))]

    aco_gerado_mes = pd.read_excel(arquivo_aco,sheet_name = 'BASE_GERAÇÃO')
    aco_gerado_mes = aco_gerado_mes[['UTD40','TURMA','ZCGTPNOTA','QTDE']]
    aco_gerado_mes.rename(columns = {'ZCGTPNOTA' : 'CARTEIRA','ZCGTPNOTA':'QTD_GERADO','UTD40':'UTD'}, inplace = True)
    aco_gerado_mes.loc[((aco_gerado_mes.TURMA == "PRÓPRIA") ), "TURMA"] = "STC"
    aco_visitado_mes.loc[((aco_visitado_mes.CARTEIRA == "CB") ), "CARTEIRA"] = "BAIXA"
    aco_visitado_mes.loc[((aco_visitado_mes.CARTEIRA == "CS") ), "CARTEIRA"] = "CORTE"
    aco_visitado_mes.loc[((aco_visitado_mes.CARTEIRA == "CA") ), "CARTEIRA"] = "RECORTE"
    aco_gerado_mes = aco_gerado_mes[((aco_gerado_mes['TURMA'] == "EPS") | (aco_visitado_mes['TURMA'] == "STC"))]


In [4]:
#aco_visitado_mes.info()
#display(aco_visitado_mes)

## (TEMPORARIO) Mapa de seleção é salvo todo dia, para se ter o histórico

In [5]:
#TROCAR O currrent_date - 1 por current_date e colocar no atena para que atualize quando clicar no botão
connection_hana = dbapi.connect(address='BRNEO695',port='30015',user=usuario_hana,password=senha_hana,databasename='BNP',sslValidateCertificate=False)
data_hoje = date.today()
data_insercao = data_hoje
cursor = connection_hana.cursor() 
query = """
    DELETE FROM CLB344173.HISTORICO_MAPA_SELECAO WHERE DATA_PEDIDO = '{var_data_insercao}';
        """
myRow = cursor.execute(query.format(var_data_insercao=data_insercao))
            
cursor = connection_hana.cursor() 
query = """
                INSERT INTO CLB344173.HISTORICO_MAPA_SELECAO SELECT PRINCIPAL.UTD, PRINCIPAL.ZONA, PRINCIPAL.CARTEIRA,PRINCIPAL.TURMA,PRINCIPAL.QTD_MIN, PRINCIPAL.QTD_MAX,PRINCIPAL.ORIGN,CAST(PRINCIPAL.QTD_CLUSTER AS INT) AS QTD_CLUSTER,PRINCIPAL.RAIO_MIN,RAIO_MAX, RAIO_INC,DIA_SEMANA,DATA_PEDIDO, QTD_SUSCETIVEIS,VALOR_COBRAVEL,PECLD,TICKET_PECLD,TICKET_COBRAVEL FROM (
SELECT 
	UTD,
	ZONA,
	CASE WHEN CARTEIRA = 'DISJUNTOR' OR CARTEIRA = 'DISJUNTOR MIX' THEN 'DISJUNTOR' 
	WHEN CARTEIRA = 'RECORTE MIX' OR CARTEIRA = 'RECORTE' THEN 'RECORTE'
	WHEN CARTEIRA = 'CONVENCIONAL' THEN 'CORTE'
	WHEN CARTEIRA = 'BAIXA MIX' OR CARTEIRA = 'BAIXA' THEN 'BAIXA' END AS CARTEIRA,
	
	TURMA,
	QTD_MIN,
	QTD_MAX,
	QTD_CLUSTERS AS ORIGN,
	CASE 
		WHEN SUBSTRING(QTD_CLUSTERS,10,1) NOT IN ('0','1','2','3','4','5','6','7','8','9') THEN 0
		WHEN DAYOFWEEK(ADD_DAYS(CURRENT_DATE,0)) = 6 THEN SUBSTRING(QTD_CLUSTERS,10,1) 
		WHEN DAYOFWEEK(ADD_DAYS(CURRENT_DATE,0)) = 5 THEN SUBSTRING(QTD_CLUSTERS,8,1) 
		WHEN DAYOFWEEK(ADD_DAYS(CURRENT_DATE,0)) = 4 THEN SUBSTRING(QTD_CLUSTERS,6,1) 
		WHEN DAYOFWEEK(ADD_DAYS(CURRENT_DATE,0)) = 3 THEN SUBSTRING(QTD_CLUSTERS,4,1)
		WHEN DAYOFWEEK(ADD_DAYS(CURRENT_DATE,0)) = 2 THEN SUBSTRING(QTD_CLUSTERS,2,1) END AS QTD_CLUSTER,
RAIO_MIN,
RAIO_MAX,
RAIO_INC,
DAYOFWEEK(ADD_DAYS(CURRENT_DATE,0)) AS DIA_SEMANA,
ADD_DAYS(CURRENT_DATE,0) AS DATA_PEDIDO
FROM CLB142840.CLUSTER_CONFIG5
) AS PRINCIPAL
LEFT JOIN (SELECT 
DATAREF,
CASE WHEN ACAO = 'DISJ' THEN 'DISJUNTOR' ELSE ACAO END AS ACAO,
UTD,ZONA,COUNT(SUSCETIVEIS.ZCGACCOUN) AS QTD_SUSCETIVEIS ,SUM(ZCGMTVCOB) AS VALOR_COBRAVEL, SUM(PECLD) AS PECLD, (SUM(PECLD)/COUNT(SUSCETIVEIS.ZCGACCOUN)) AS TICKET_PECLD, (SUM(ZCGMTVCOB)/COUNT(SUSCETIVEIS.ZCGACCOUN)) AS TICKET_COBRAVEL FROM CLB961920.SUSCETIVEIS
LEFT JOIN (SELECT ZCGVKONT AS ZCGACCOUN, SUM(ZCGPECLDALOC) AS PECLD
FROM CLB_CCS_ICC.ZCT_DS_CARGA_AGING_COELBA
GROUP BY ZCGVKONT) AS PDD
ON PDD.ZCGACCOUN = SUSCETIVEIS.ZCGACCOUN
WHERE ZONA IS NOT NULL
GROUP BY DATAREF,ACAO,UTD,ZONA) AS DADOS_FINANCEIROS
ON DADOS_FINANCEIROS.DATAREF = DATA_PEDIDO AND DADOS_FINANCEIROS.UTD = PRINCIPAL.UTD AND DADOS_FINANCEIROS.ZONA = PRINCIPAL.ZONA AND ACAO = CARTEIRA
                """
myRow = cursor.execute(query.format(var_data_insercao=data_insercao))

#### 3 Coletar dados SAP HANA base geração/seleção

In [6]:
connection_hana = dbapi.connect(address='BRNEO695',port='30015',user=usuario_hana,password=senha_hana,databasename='BNP',sslValidateCertificate=False)
sqll = """
SELECT UTD, ZONA,TURMA, CARTEIRA,DATA_PEDIDO, COUNT(ZCGACCOUN) AS NOTAS_ESPERADAS_ATENA,COUNT(NOTA) AS NOTAS_GERADAS_TAB4,COUNT(ZCGACCOUN) -COUNT(NOTA) AS DEFASAGEM_GERACAO, DIA_SEMANA,FLAG_DEF FROM (
SELECT 
	AH.ZCGACCOUN,
	ZCGUTD AS UTD,
	ZCGMTVCOB,
	CASE WHEN CARTEIRA = 'DISJUNTOR' OR CARTEIRA = 'DISJUNTOR MIX' THEN 'DISJUNTOR' 
	WHEN CARTEIRA = 'RECORTE MIX' OR CARTEIRA = 'RECORTE' THEN 'RECORTE'
	WHEN CARTEIRA = 'CONVENCIONAL' THEN 'CORTE'
	WHEN CARTEIRA = 'BAIXA MIX' OR CARTEIRA = 'BAIXA' THEN 'BAIXA' END AS CARTEIRA,
	TURMA,
	DATA_PEDIDO,
	DAYOFWEEK(DATA_PEDIDO) AS DIA_SEMANA,
	ZCGIDZONA AS ZONA,
	FLAG_DEF,
	NOTA,
	TIPO_NOTA
FROM CLB961920.ATENA_HIST AH
LEFT JOIN (SELECT TAB004.ZCGQMNUM AS NOTA,
	TAB004.ZCGDTCRIA AS ZCGDTCRIA,
	TAB004.ZCGINSTAL AS INSTALACAO,
	TAB004.ZCGPEDIDO AS PEDIDO,
	TAB004.ZCGACCOUN,
	TAB004.ZCGTPNOTA AS TIPO_NOTA
FROM CLB_CCS_ICC.ZCT_DS_TAB004 TAB004
WHERE ZCGTPNOTA IN ('CA','CS','CB')) TAB004
ON TAB004.ZCGACCOUN = AH.ZCGACCOUN AND 
(ZCGDTCRIA >= ADD_DAYS(DATA_PEDIDO, 0) AND ZCGDTCRIA <= ADD_DAYS(DATA_PEDIDO, 1) )
WHERE DATA_PEDIDO > '2022-08-01'
)
GROUP BY UTD, ZONA, DATA_PEDIDO,TURMA, CARTEIRA,DIA_SEMANA,FLAG_DEF
            """
base_selecao = pd.read_sql(sqll,connection_hana)
base_selecao.to_csv('base_selecao.csv',index=False,sep=';',decimal=',')

#### 4 Coletar dados SAP HANA esperado do mapa de seleção VS inserido no atena_hist

In [7]:
sql2 = """
SELECT UTD, ZONA, CARTEIRA,TURMA,TOTAL_ESPERADO_MAPA,TOTAL_ESPERADO_ATENA, DATA_PEDIDO, DIA_SEMANA,FLAG_DEF FROM (
SELECT HMS.UTD, HMS.ZONA, HMS.CARTEIRA,HMS.TURMA,SUM(TOTAL_ESPERADO_MAPA) AS TOTAL_ESPERADO_MAPA,NULL AS TOTAL_ESPERADO_ATENA, HMS.DATA_PEDIDO, DAYOFWEEK(DATA_PEDIDO) AS DIA_SEMANA, NULL AS FLAG_DEF

FROM (
SELECT HMS.UTD,ZONA,CARTEIRA,TURMA,QTD_CLUSTER* QTD_MIN AS TOTAL_ESPERADO_MAPA,DATA_PEDIDO FROM CLB344173.HISTORICO_MAPA_SELECAO HMS
) HMS
WHERE  HMS.ZONA LIKE ('%0%') OR  HMS.ZONA LIKE ('%1%') OR HMS.ZONA  LIKE ('%2%') OR  HMS.ZONA LIKE ('%3%') OR  HMS.ZONA LIKE ('%4%')
GROUP BY HMS.UTD, HMS.ZONA, HMS.CARTEIRA,HMS.TURMA,HMS.DATA_PEDIDO

UNION 

SELECT UTD, ZONA,CARTEIRA,TURMA, NULL AS TOTAL_ESPERADO_MAPA,COUNT(ZCGACCOUN) AS TOTAL_ESPERADO_ATENA, DATA_PEDIDO, DIA_SEMANA,FLAG_DEF FROM (
SELECT 
	AH.ZCGACCOUN,
	ZCGUTD AS UTD,
	ZCGMTVCOB,
	CASE WHEN CARTEIRA = 'CONVENCIONAL' THEN 'CORTE'
	WHEN CARTEIRA = 'RECORTE' THEN CARTEIRA
	WHEN CARTEIRA = 'BAIXA MIX' OR CARTEIRA = 'BAIXA'  THEN 'BAIXA'
	WHEN CARTEIRA = 'DISJUNTOR' OR CARTEIRA = 'DISJUNTOR MIX' THEN 'DISJUNTOR' END AS CARTEIRA,
	TURMA,
	DATA_PEDIDO,
	DAYOFWEEK(DATA_PEDIDO) AS DIA_SEMANA,
	ZCGIDZONA AS ZONA,
	FLAG_DEF
FROM CLB961920.ATENA_HIST AH
WHERE DATA_PEDIDO > '2022-07-12'
AND FLAG_DEF = 'N'
)
GROUP BY UTD, ZONA, DATA_PEDIDO,TURMA, CARTEIRA,DIA_SEMANA,FLAG_DEF)
WHERE DATA_PEDIDO >= '2022-08-18'
AND (TOTAL_ESPERADO_ATENA>0 OR TOTAL_ESPERADO_MAPA>0)
            """
base_mapa_atena = pd.read_sql(sql2,connection_hana)
#base_pecld_mun.to_csv('consulta_hemera.csv',index=False,sep=';',decimal=',')
#usuario = 'u344173'

base_mapa_atena.to_csv('base_mapa_atena.csv',index=False,sep=';',decimal=',')

#### 5 Adicionar dados Defasagem

In [8]:
#inserir o check se existe aba
df_pedido_defasagem = pd.read_excel(arquivo_defasagem,sheet_name = aba)
df_pedido_defasagem = df_pedido_defasagem[['SETOR','UTD','ID','TIPO','QTD. TURMAS','QTD. NOTAS/TURMA','OBS']]
df_pedido_defasagem.rename(columns = {'ID' : 'ZONA','TIPO':'TIPO_SERVICO','QTD. TURMAS':'QTD_TURMAS','QTD. NOTAS/TURMA':'QTD_NOTAS','OBS':'JUSTIFICATIVA'}, inplace = True)
df_pedido_defasagem['QTD_TURMAS'] = df_pedido_defasagem['QTD_TURMAS'].fillna(0)
df_pedido_defasagem['QTD_NOTAS'] = df_pedido_defasagem['QTD_NOTAS'].fillna(0)
df_pedido_defasagem['TIPO_SERVICO'] = df_pedido_defasagem['TIPO_SERVICO'].fillna('-')
df_pedido_defasagem['JUSTIFICATIVA'] = df_pedido_defasagem['JUSTIFICATIVA'].fillna('-')
df_pedido_defasagem['QTD_TURMAS'] = df_pedido_defasagem['QTD_TURMAS'].astype(int)
df_pedido_defasagem['QTD_NOTAS'] = df_pedido_defasagem['QTD_NOTAS'].astype(int)
df_pedido_defasagem['ZONA']  = df_pedido_defasagem['ZONA'].astype(str).str.pad(4,'left','0')
df_pedido_defasagem['ACAO'] = ""
df_pedido_defasagem['CARTEIRA'] = ""
df_pedido_defasagem['TURMA'] = "STC"
df_pedido_defasagem.loc[((df_pedido_defasagem.JUSTIFICATIVA == "-") &(df_pedido_defasagem.TIPO_SERVICO == "-") & (df_pedido_defasagem.QTD_TURMAS == 0) & (df_pedido_defasagem.QTD_NOTAS == 0)), "ACAO"] = "APAGAR"
df_pedido_defasagem = df_pedido_defasagem[((df_pedido_defasagem['ACAO'] != "APAGAR"))]
df_pedido_defasagem.loc[(df_pedido_defasagem.TIPO_SERVICO == "CA"), "CARTEIRA"] = "RECORTE"
df_pedido_defasagem.loc[(df_pedido_defasagem.TIPO_SERVICO == "CS"), "CARTEIRA"] = "CORTE"
df_pedido_defasagem['DEFASAGEM_PROGRAMACAO'] = df_pedido_defasagem['QTD_TURMAS'] * df_pedido_defasagem['QTD_NOTAS']
df_pedido_defasagem = df_pedido_defasagem.dropna().reset_index(drop=True)


In [9]:
data_hoje = date.today()
data_insercao = data_hoje
dia_semana = datetime.datetime.today().weekday()
if dia_semana == 0: #segunda
    data_pedido = date.today() - timedelta(3)
elif dia_semana == 6: #domingo
    data_pedido = date.today() - timedelta(3)
elif dia_semana == 5: #sabado
    data_pedido = date.today() - timedelta(2)
else:
    data_pedido = date.today() - timedelta(1)
cursor = connection_hana.cursor() 
query = """
    DELETE FROM CLB344173.HISTORICO_SOLICITACAO_DEFASAGEM 
    WHERE DATA_INSERCAO = '{var_data_insercao}'
    """
myRow = cursor.execute(query.format(var_data_insercao=data_insercao))
for i in range(len(df_pedido_defasagem.index)):
    setor = str(df_pedido_defasagem["SETOR"][i])
    utd = str(df_pedido_defasagem["UTD"][i])
    zona = str(df_pedido_defasagem["ZONA"][i])
    tipo_servico = str(df_pedido_defasagem["TIPO_SERVICO"][i])
    qtd_turmas = int(df_pedido_defasagem["QTD_TURMAS"][i])
    qtd_notas = int(df_pedido_defasagem["QTD_NOTAS"][i])
    justificativa = str(df_pedido_defasagem["JUSTIFICATIVA"][i])

    insert_list = [(setor,utd,zona,tipo_servico,qtd_turmas,qtd_notas,justificativa,data_pedido,data_insercao)]
    cursor.executemany("""
    INSERT INTO CLB344173.HISTORICO_SOLICITACAO_DEFASAGEM ("SETOR","UTD","ZONA","TIPO_SERVICO","QTD_TURMAS","QTD_NOTAS","JUSTIFICATIVA","DATA_PEDIDO","DATA_INSERCAO")
    VALUES(?,?,?,?,?,?,?,?,?)
    """,insert_list)

#### 6. Construção da base final para análise da necessidade de defasagem do dia

In [11]:
sql3 = """
    SELECT UTD,ZONA,TURMA, CARTEIRA,DATA_PEDIDO, COUNT(ZCGACCOUN) AS NOTAS_ESPERADAS_ATENA,COUNT(NOTA) AS NOTAS_GERADAS_TAB4,COUNT(ZCGACCOUN) -COUNT(NOTA) AS DEFASAGEM_GERACAO FROM (
SELECT 
	AH.ZCGACCOUN,
	ZCGUTD AS UTD,
	ZCGMTVCOB,
	CASE WHEN CARTEIRA = 'DISJUNTOR' OR CARTEIRA = 'DISJUNTOR MIX' THEN 'DISJUNTOR' 
	WHEN CARTEIRA = 'RECORTE MIX' OR CARTEIRA = 'RECORTE' THEN 'RECORTE'
	WHEN CARTEIRA = 'CONVENCIONAL' THEN 'CORTE'
	WHEN CARTEIRA = 'BAIXA MIX' OR CARTEIRA = 'BAIXA' THEN 'BAIXA' END AS CARTEIRA,
	TURMA,
	DATA_PEDIDO,
	DAYOFWEEK(DATA_PEDIDO) AS DIA_SEMANA,
	ZCGIDZONA AS ZONA,
	FLAG_DEF,
	NOTA,
	TIPO_NOTA
FROM CLB961920.ATENA_HIST AH
LEFT JOIN (SELECT TAB004.ZCGQMNUM AS NOTA,
	TAB004.ZCGDTCRIA AS ZCGDTCRIA,
	TAB004.ZCGINSTAL AS INSTALACAO,
	TAB004.ZCGPEDIDO AS PEDIDO,
	TAB004.ZCGACCOUN,
	TAB004.ZCGTPNOTA AS TIPO_NOTA
FROM CLB_CCS_ICC.ZCT_DS_TAB004 TAB004
WHERE ZCGTPNOTA IN ('CA','CS','CB')) TAB004
ON TAB004.ZCGACCOUN = AH.ZCGACCOUN AND 
(ZCGDTCRIA >= ADD_DAYS(DATA_PEDIDO, 0) AND ZCGDTCRIA <= ADD_DAYS(DATA_PEDIDO, 1) )
WHERE DATA_PEDIDO > '2022-07-25'

UNION 

SELECT 
	CH.ZCGACCOUN,
	ZCGUTD AS UTD,
	ZCGMTVCOB,
	CASE WHEN CARTEIRA = 'BAIXA MIX' OR CARTEIRA = 'BAIXA' THEN 'BAIXA' 
	WHEN CARTEIRA = 'RECORTE MIX' OR CARTEIRA = 'RECORTE' THEN 'RECORTE'
	WHEN CARTEIRA = 'DISJUNTOR' OR CARTEIRA = 'DISJ MIX' THEN 'DISJUNTOR'
	WHEN CARTEIRA = 'CONVENCIONAL' THEN 'CORTE'
	WHEN (CARTEIRA = 'TOP5' OR CARTEIRA = 'TOP_STC')AND DAYOFWEEK(DATA_PEDIDO) = 4 THEN 'RECORTE'
	WHEN (CARTEIRA = 'TOP5' OR CARTEIRA = 'TOP_STC') AND DAYOFWEEK(DATA_PEDIDO) <> 4 THEN 'CORTE'
	WHEN (CARTEIRA = 'PROPRIA') AND DAYOFWEEK(DATA_PEDIDO) = 4 THEN 'RECORTE'
	WHEN (CARTEIRA = 'PROPRIA') AND DAYOFWEEK(DATA_PEDIDO) <> 4 THEN 'CORTE' END AS CARTEIRA,
	
	CASE WHEN CARTEIRA = 'BAIXA MIX' OR CARTEIRA = 'BAIXA' THEN 'EPS' 
	WHEN CARTEIRA = 'RECORTE MIX' OR CARTEIRA = 'RECORTE' THEN 'EPS'
	WHEN CARTEIRA = 'DISJ MIX' OR CARTEIRA = 'DISJUNTOR' THEN 'EPS'
	WHEN CARTEIRA = 'CONVENCIONAL' THEN 'EPS'
	WHEN (CARTEIRA = 'TOP5' ) THEN 'EPS'
	WHEN ( CARTEIRA = 'TOP_STC')  THEN 'STC'
	WHEN (CARTEIRA = 'PROPRIA') THEN 'STC' END AS TURMA,
	DATA_PEDIDO,
	DAYOFWEEK(DATA_PEDIDO) AS DIA_SEMANA,
	ZCGIDZONA AS ZONA,
	FLAG_DEF,
	NOTA,
	TIPO_NOTA
FROM CLB142840.CLUSTER_HIST CH
LEFT JOIN (SELECT TAB004.ZCGQMNUM AS NOTA,
	TAB004.ZCGDTCRIA AS ZCGDTCRIA,
	TAB004.ZCGINSTAL AS INSTALACAO,
	TAB004.ZCGPEDIDO AS PEDIDO,
	TAB004.ZCGACCOUN,
	TAB004.ZCGTPNOTA AS TIPO_NOTA
FROM CLB_CCS_ICC.ZCT_DS_TAB004 TAB004
WHERE ZCGTPNOTA IN ('CA','CS','CB')) TAB004
ON TAB004.ZCGACCOUN = CH.ZCGACCOUN AND 
(ZCGDTCRIA >= ADD_DAYS(DATA_PEDIDO, 0) AND ZCGDTCRIA <= ADD_DAYS(DATA_PEDIDO, 1) )
WHERE DATA_PEDIDO > '2022-07-25'
AND CARTEIRA <> 'PRONTIDAO'
AND CH.ZCGACCOUN NOT IN (SELECT DISTINCT AH.ZCGACCOUN FROM CLB961920.ATENA_HIST AH WHERE DATA_PEDIDO = '{}')
AND DATA_PEDIDO = '{}'
AND FLAG_DEF = 'N'
)
WHERE FLAG_DEF = 'N'
AND DATA_PEDIDO = '{}'
GROUP BY UTD, ZONA, DATA_PEDIDO,TURMA, CARTEIRA
                                    """.format(data_pedido,data_pedido,data_pedido)

sql4 = """
SELECT ZONA, TURMA,CARTEIRA,TOTAL_ESPERADO_MAPA, DATA_PEDIDO FROM (
SELECT HMS.UTD, HMS.ZONA, HMS.CARTEIRA,HMS.TURMA,SUM(TOTAL_ESPERADO_MAPA) AS TOTAL_ESPERADO_MAPA,NULL AS TOTAL_ESPERADO_ATENA, HMS.DATA_PEDIDO, DAYOFWEEK(DATA_PEDIDO) AS DIA_SEMANA, NULL AS FLAG_DEF

FROM (
SELECT HMS.UTD,ZONA,CARTEIRA,TURMA,QTD_CLUSTER* QTD_MAX AS TOTAL_ESPERADO_MAPA,DATA_PEDIDO FROM CLB344173.HISTORICO_MAPA_SELECAO HMS
) HMS
WHERE  HMS.ZONA LIKE ('%0%') OR  HMS.ZONA LIKE ('%1%') OR HMS.ZONA  LIKE ('%2%') OR  HMS.ZONA LIKE ('%3%') OR  HMS.ZONA LIKE ('%4%')
GROUP BY HMS.UTD, HMS.ZONA, HMS.CARTEIRA,HMS.TURMA,HMS.DATA_PEDIDO
)
WHERE DATA_PEDIDO = '{}'
 """.format(data_pedido)

sql5 = """
SELECT 
ZONA, 
CASE WHEN ACAO = 'DISJ' THEN 'DISJUNTOR' ELSE ACAO END AS CARTEIRA,
COUNT(SUSCETIVEIS.ZCGACCOUN) AS QTD_SUSCETIVEIS ,SUM(ZCGMTVCOB) AS VALOR_COBRAVEL, SUM(PECLD) AS PECLD, (SUM(PECLD)/COUNT(SUSCETIVEIS.ZCGACCOUN)) AS TICKET_PECLD, (SUM(ZCGMTVCOB)/COUNT(SUSCETIVEIS.ZCGACCOUN)) AS TICKET_COBRAVEL FROM CLB961920.SUSCETIVEIS
LEFT JOIN (SELECT ZCGVKONT AS ZCGACCOUN, SUM(ZCGPECLDALOC) AS PECLD
FROM CLB_CCS_ICC.ZCT_DS_CARGA_AGING_COELBA
GROUP BY ZCGVKONT) AS PDD
ON PDD.ZCGACCOUN = SUSCETIVEIS.ZCGACCOUN
WHERE ZONA IS NOT NULL
AND DATAREF = ADD_DAYS(CURRENT_DATE,0) 
GROUP BY DATAREF,ACAO,UTD,ZONA
"""


df_defasagem_geracao_hoje = pd.read_sql(sql3,connection_hana)

df_defasagem_mapa_hoje = pd.read_sql(sql4,connection_hana)

df_suscetiveis_hoje = pd.read_sql(sql5,connection_hana)

df_defasagem_geracao_hoje['ZONA'] = df_defasagem_geracao_hoje['ZONA'].fillna(0.0)
df_defasagem_geracao_hoje['ZONA']  = df_defasagem_geracao_hoje['ZONA'].astype(int)
df_defasagem_geracao_hoje['ZONA']  = df_defasagem_geracao_hoje['ZONA'].astype(str).str.pad(4,'left','0')

In [ ]:
print(dia_semana)

In [12]:
df_pedido_defasagem = df_pedido_defasagem[['ZONA','DEFASAGEM_PROGRAMACAO','QTD_TURMAS','QTD_NOTAS','JUSTIFICATIVA','CARTEIRA','TURMA']]
df_pedido_defasagem.info()
base_defasagens = pd.merge(df_defasagem_geracao_hoje, df_defasagem_mapa_hoje, how = 'outer', on=['ZONA','TURMA','CARTEIRA','DATA_PEDIDO'])
base_final = pd.merge(df_suscetiveis_hoje, base_defasagens, how = 'outer', on=['ZONA','CARTEIRA'])
base_final = pd.merge(base_final, df_pedido_defasagem, how = 'outer', on=['ZONA','CARTEIRA','TURMA'])
base_final.loc[((base_final.NOTAS_ESPERADAS_ATENA >0) |(base_final.NOTAS_GERADAS_TAB4 > 0) | (base_final.DEFASAGEM_GERACAO > 0) | (base_final.TOTAL_ESPERADO_MAPA > base_final.NOTAS_ESPERADAS_ATENA) | (base_final.DEFASAGEM_PROGRAMACAO >= 0)), "ACAO"] = "MANTER"
base_final = base_final[((base_final['ACAO'] == "MANTER"))]
base_final = base_final[['ZONA','TURMA','CARTEIRA','TOTAL_ESPERADO_MAPA','NOTAS_ESPERADAS_ATENA','NOTAS_GERADAS_TAB4','DEFASAGEM_GERACAO','DEFASAGEM_PROGRAMACAO','QTD_TURMAS','QTD_NOTAS','JUSTIFICATIVA','QTD_SUSCETIVEIS','VALOR_COBRAVEL','PECLD','TICKET_COBRAVEL','TICKET_PECLD']]
base_final['DEFASAGEM_MAPA'] = base_final['TOTAL_ESPERADO_MAPA'] - base_final['NOTAS_ESPERADAS_ATENA']
base_final = base_final[['ZONA','TURMA','CARTEIRA','TOTAL_ESPERADO_MAPA','NOTAS_ESPERADAS_ATENA','NOTAS_GERADAS_TAB4','DEFASAGEM_GERACAO','DEFASAGEM_MAPA','DEFASAGEM_PROGRAMACAO','QTD_TURMAS','QTD_NOTAS','JUSTIFICATIVA','QTD_SUSCETIVEIS','VALOR_COBRAVEL','PECLD','TICKET_COBRAVEL','TICKET_PECLD']]
base_final.to_csv('base_defasagem_hoje.csv',index=False,sep=';',decimal=',',encoding='utf-8-sig')
base_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   ZONA                   21 non-null     object
 1   DEFASAGEM_PROGRAMACAO  21 non-null     int32 
 2   QTD_TURMAS             21 non-null     int32 
 3   QTD_NOTAS              21 non-null     int32 
 4   JUSTIFICATIVA          21 non-null     object
 5   CARTEIRA               21 non-null     object
 6   TURMA                  21 non-null     object
dtypes: int32(3), object(4)
memory usage: 1.0+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 515 entries, 1 to 3242
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ZONA                   515 non-null    object 
 1   TURMA                  515 non-null    object 
 2   CARTEIRA               515 non-null    object 
 3   TOTAL_ESPERADO_MAP

In [ ]:
cursor = connection_hana.cursor() 
query = """
    DELETE FROM CLB344173.HISTORICO_SOLICITACAO_DEFASAGEM_FINAL
    WHERE DATA_INSERCAO = '{var_data_insercao}'
    """
myRow = cursor.execute(query.format(var_data_insercao=data_insercao))

for i in range(len(base_final.index)):
    zona = str(base_final["ZONA"][i])
    turma = str(base_final["TURMA"][i])
    carteira = str(base_final["CARTEIRA"][i])
    total_esperado_mapa = int(base_final["TOTAL_ESPERADO_MAPA"][i])
    total_esperadas_atena = int(base_final["NOTAS_ESPERADAS_ATENA"][i])
    notas_geradas_tab4 = int(base_final["NOTAS_GERADAS_TAB4"][i])
    defasagem_geracao = int(base_final["DEFASAGEM_GERACAO"][i])
    defasagem_mapa = int(base_final["DEFASAGEM_MAPA"][i])
    defasagem_programacao = int(base_final["DEFASAGEM_PROGRAMACAO"][i])
    qtd_turmas = int(base_final["QTD_TURMAS"][i])
    qtd_notas = int(base_final["QTD_NOTAS"][i])
    justificativa = str(base_final["JUSTIFICATIVA"][i])
    qtd_suscetiveis = int(base_final["QTD_SUSCETIVEIS"][i])
    valor_cobravel = float(base_final["VALOR_COBRAVEL"][i])
    pecld = float(base_final["PECLD"][i])
    ticket_cobravel = float(base_final["TICKET_COBRAVEL"][i])
    ticket_pecld = float(base_final["TICKET_PECLD"][i])
    
    

    insert_list = [(zona,turma,carteira,total_esperado_mapa,total_esperadas_atena,notas_geradas_tab4,defasagem_geracao,defasagem_mapa,defasagem_programacao,qtd_turmas,qtd_notas,justificativa,qtd_suscetiveis,valor_cobravel,pecld,ticket_cobravel,ticket_pecld,data_insercao)]
    cursor.executemany("""
    INSERT INTO CLB344173.HISTORICO_SOLICITACAO_DEFASAGEM_FINAL ("ZONA","TURMA","CARTEIRA","TOTAL_ESPERADO_MAPA","TOTAL_ESPERADAS_ATENA","NOTAS_GERADAS_TAB4","DEFASAGEM_GERACAO","DEFASAGEM_MAPA","DEFASAGEM_PROGRAMACAO","QTD_TURMAS","QTD_NOTAS","JUSTIFICATIVA","QTD_SUSCETIVEIS","VALOR_COBRAVEL","PECLD","TICKET_COBRAVEL","TICKET_PECLD","DATA_INSERCAO")
    VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
    """,insert_list)

In [ ]:
display(df_defasagem_geracao_hoje)

In [ ]:
for i in range(len(df_pedido_defasagem.index)):
    total_esperado_mapa = int(df_pedido_defasagem["TOTAL_ESPERADO_MAPA"][i])
    print(total_esperado_mapa)